In [3]:
import json


In [12]:
def convert_to_yolo(json_data, class_mapping):
    yolo_annotations = {}

    for image_key, image_data in json_data.items():
        filename = image_data["filename"]
        width = image_data["file_attributes"].get("width", 1)  # Default width to 1 if not specified
        height = image_data["file_attributes"].get("height", 1)  # Default height to 1 if not specified

        annotations = []
        for region in image_data["regions"]:
            shape_attributes = region["shape_attributes"]
            region_attributes = region["region_attributes"]

            x = shape_attributes["x"]
            y = shape_attributes["y"]
            w = shape_attributes["width"]
            h = shape_attributes["height"]

            class_name = region_attributes["dvla"]
            class_index = class_mapping.get(class_name)

            if class_index is not None:
                x_normalized = x / width
                y_normalized = y / height
                w_normalized = w / width
                h_normalized = h / height

                annotations.append(f"{class_index} {x_normalized} {y_normalized} {w_normalized} {h_normalized}")

        yolo_annotations[filename] = annotations

    return yolo_annotations

In [15]:
json_path = "dataset/eval/annotations.json"
class_mapping = {"firstname": 0, "surname": 1, "dob": 2, "issue date": 3, "expiry date": 4,
                 "authority": 5, "driver no": 6, "signature": 7, "address": 8, "entitlements": 9}

with open(json_path, "r") as json_file:
    json_data = json.load(json_file)

yolo_annotations = convert_to_yolo(json_data, class_mapping)

# Save YOLO annotations to a single text file for each image
for filename, annotations in yolo_annotations.items():
    with open(f"{filename}.txt", "w") as yolo_file:
        for annotation in annotations:
            yolo_file.write(annotation + '\n')